In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,experimental
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf

import numpy as np
import pandas as pd
import os

import cv2
import matplotlib.pyplot as plt

In [2]:
train_path="train"
test_path="test"
val_path="validate"

In [3]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

x_test=[]
for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)
        
        img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [4]:
# Generate Lable 
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:

training_set = train_datagen.flow_from_directory("train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 149 images belonging to 5 classes.
Found 69 images belonging to 5 classes.
Found 67 images belonging to 5 classes.


In [6]:
train_y = training_set.classes
test_y = test_set.classes
val_y = val_set.classes

In [7]:
train_y = training_set.classes
test_y = test_set.classes
val_y = val_set.classes

In [9]:
IMG_SIZE = 224
def augment(image, label):
    img = tf.cast(image, tf.float32)
    img = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    img = (image / 255.0)
    img = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
    img = tf.image.random_brightness(image, max_delta=0.5)
    return img, label  

In [10]:
#Data Augmentation
augmented_images = list(map(augment,x_train,train_y))

In [11]:
for image in augmented_images:
    x_train.append(image[0])
    train_y = np.append(train_y,image[1])

In [13]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [14]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [15]:
lables = {}
lables[0] = "bridge"
lables[1] = "brokenPipe"
lables[2] = "fallenPole"
lables[3] = "garbage"
lables[4] = "pitHole"
lables

{0: 'bridge', 1: 'brokenPipe', 2: 'fallenPole', 3: 'garbage', 4: 'pitHole'}

In [27]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
base_model.trainable = True

In [28]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (224,224,3), name='inputLayer')

x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(5, name='outputLayer')(x)
outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")


In [29]:
model.summary()

Model: "FeatureExtractionModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputLayer (InputLayer)      [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
poolingLayer (GlobalAverageP (None, 1280)              0         
_________________________________________________________________
outputLayer (Dense)          (None, 5)                 6405      
_________________________________________________________________
activationLayer (Activation) (None, 5)                 0         
Total params: 4,055,976
Trainable params: 4,013,953
Non-trainable params: 42,023
_________________________________________________________________


In [30]:

for layer in model.layers[1].layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

In [31]:
for lnum, layer in enumerate(model.layers):
    print(lnum, layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

0 inputLayer True float32 <Policy "float32">
1 efficientnetb0 True float32 <Policy "float32">
2 poolingLayer True float32 <Policy "float32">
3 outputLayer True float32 <Policy "float32">
4 activationLayer True float32 <Policy "float32">


In [32]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ["accuracy"])
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [34]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)


Epoch 1/10
14/14 [==============================] - 129s 10s/step - loss: 1.5814 - accuracy: 0.2886 - val_loss: 1.6175 - val_accuracy: 0.2239
Epoch 2/10
14/14 [==============================] - 59s 4s/step - loss: 1.5802 - accuracy: 0.2886 - val_loss: 1.6196 - val_accuracy: 0.2239
Epoch 3/10
14/14 [==============================] - 61s 4s/step - loss: 1.5801 - accuracy: 0.2886 - val_loss: 1.6232 - val_accuracy: 0.2239
Epoch 4/10
14/14 [==============================] - 57s 4s/step - loss: 1.5799 - accuracy: 0.2886 - val_loss: 1.6158 - val_accuracy: 0.2239
Epoch 5/10
14/14 [==============================] - 58s 4s/step - loss: 1.5832 - accuracy: 0.2886 - val_loss: 1.6216 - val_accuracy: 0.2239
Epoch 6/10
14/14 [==============================] - 57s 4s/step - loss: 1.5832 - accuracy: 0.2886 - val_loss: 1.6100 - val_accuracy: 0.2239
Epoch 7/10
14/14 [==============================] - 58s 4s/step - loss: 1.5818 - accuracy: 0.2886 - val_loss: 1.6198 - val_accuracy: 0.2239
Epoch 8/10
14/14 [